In [1]:
import numpy as np 
import pandas as pd
import os
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader,Dataset, sampler, random_split
from torchvision import models
!pip install timm # kaggle doesnt have it installed by default
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from tqdm import tqdm
import time
import copy
from os import listdir
from os.path import isfile, join
from PIL import Image
import glob 
import math
from typing import Any, Callable, cast, Dict, List, Optional, Tuple, Union


In [2]:
#Método para obtener todas las especies de pájaros a partir de la estructura de carpetas
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [3]:
def img_to_patch(x, patch_size, flatten_channels=True):
    """
    Inputs:
        x - torch.Tensor representing the image of shape [B, C, H, W]
        patch_size - Number of pixels per dimension of the patches (integer)
        flatten_channels - If True, the patches will be returned in a flattened format
                           as a feature vector instead of a image grid.
    """
    B, C, H, W = x.shape
    x = x.reshape(B, C, H//patch_size, patch_size, W//patch_size, patch_size)
    x = x.permute(0, 2, 4, 1, 3, 5) # [B, H', W', C, p_H, p_W]
    x = x.flatten(1,2)              # [B, H'*W', C, p_H, p_W]
    if flatten_channels:
        x = x.flatten(2,4)          # [B, H'*W', C*p_H*p_W]    
    return x



In [4]:
def pre_image(image_path,model):
    #img = Image.open(image_path)
    #mean = [0.485, 0.456, 0.406] 
    #std = [0.229, 0.224, 0.225]
    #transform_norm = transforms.Compose([transforms.ToTensor(), 
    #transforms.Resize((128,128))])
    transform_norm = transforms.ToTensor()
   # get normalized image
    img_normalized = transform_norm(image_path).float()
    img_normalized = img_normalized.unsqueeze(0)
   # input = Variable(image_tensor)
    img_normalized = img_normalized.to(device)
   # print(img_normalized.shape)
    with torch.no_grad():
        model.eval()
        output =model(img_normalized)
     # print(output)
        index = output.data.cpu().numpy().argmax()
        class_name = classes[index]
        return class_name

In [5]:
IMG_EXTENSIONS = (".jpg", ".jpeg", ".png", ".ppm", ".bmp", ".pgm", ".tif", ".tiff", ".webp")

class ImageFolderCustom(datasets.DatasetFolder):
    
    def __init__(
        self,
        root: str,
        setname: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        loader: Callable[[str], Any] = datasets.folder.default_loader,
        is_valid_file: Optional[Callable[[str], bool]] = None,
    ):
        super().__init__(
            root,
            loader,
            IMG_EXTENSIONS if is_valid_file is None else None,
            transform=transform,
            target_transform=target_transform,
            is_valid_file=is_valid_file,
        )
        
        classes, class_to_idx = self.find_classes(self.root)
        self.samples = self.make_dataset2(self.root,setname, class_to_idx, IMG_EXTENSIONS, is_valid_file)
        self.imgs = self.samples
        
    @staticmethod
    def make_dataset2(
        directory: str,
        setname: str,
        class_to_idx: Optional[Dict[str, int]] = None,
        extensions: Optional[Union[str, Tuple[str, ...]]] = None,
        is_valid_file: Optional[Callable[[str], bool]] = None,    
    ) -> List[Tuple[str, int]]:
        """
        Generates a list of samples of a form (path_to_sample, class).

        See :class:`DatasetFolder` for details.

        Note: The class_to_idx parameter is here optional and will use the logic of the ``find_classes`` function
        by default.
        """
        setname = setname
        assert setname in ['train','val']

        if class_to_idx is None:
            _, class_to_idx = find_classes(directory)
        elif not class_to_idx:
            raise ValueError("'class_to_index' must have at least one entry to collect any samples.")

        both_none = extensions is None and is_valid_file is None
        both_something = extensions is not None and is_valid_file is not None
        if both_none or both_something:
            raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")

        if extensions is not None:

            def is_valid_file(x: str) -> bool:
                return datasets.folder.has_file_allowed_extension(x, extensions)  # type: ignore[arg-type]

        is_valid_file = cast(Callable[[str], bool], is_valid_file)

        instances = []
        available_classes = set()
        for target_class in sorted(class_to_idx.keys()):
            class_index = class_to_idx[target_class]
            target_dir = os.path.join(directory, target_class)
            if not os.path.isdir(target_dir):
                continue
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                num_images=len(fnames)
                num_separator=math.ceil(num_images*0.9)
                i=0
                #print(i, num_separator)
                #print(setname=='train' and i>=0 and i<num_separator)
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    #print(i)
                    if(setname=='train' and i>=0 and i<num_separator or (setname=='val' and i>=num_separator and i<num_images)):
                        if is_valid_file(path):
                            item = path, class_index
                            instances.append(item)
                            if target_class not in available_classes:
                                available_classes.add(target_class)
                    i=i+1
    
        empty_classes = set(class_to_idx.keys()) - available_classes
        if empty_classes:
            msg = f"Found no valid file for the classes {', '.join(sorted(empty_classes))}. "
            if extensions is not None:
                msg += f"Supported extensions are: {extensions if isinstance(extensions, str) else ', '.join(extensions)}"
            raise FileNotFoundError(msg)
    
        return instances

In [6]:
class AttentionBlock(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of input and attention feature vectors
            hidden_dim - Dimensionality of hidden layer in feed-forward network
                         (usually 2-4x larger than embed_dim)
            num_heads - Number of heads to use in the Multi-Head Attention block
            dropout - Amount of dropout to apply in the feed-forward network
        """
        super().__init__()

        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads,
                                          dropout=dropout)
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )


    def forward(self, x):
        inp_x = self.layer_norm_1(x)
        x = x + self.attn(inp_x, inp_x, inp_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x

In [7]:
class VisionTransformer(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of the input feature vectors to the Transformer
            hidden_dim - Dimensionality of the hidden layer in the feed-forward networks
                         within the Transformer
            num_channels - Number of channels of the input (3 for RGB)
            num_heads - Number of heads to use in the Multi-Head Attention block
            num_layers - Number of layers to use in the Transformer
            num_classes - Number of classes to predict
            patch_size - Number of pixels that the patches have per dimension
            num_patches - Maximum number of patches an image can have
            dropout - Amount of dropout to apply in the feed-forward network and
                      on the input encoding
        """
        super().__init__()

        self.patch_size = patch_size

        # Layers/Networks
        self.input_layer = nn.Linear(num_channels*(patch_size**2), embed_dim)
        self.transformer = nn.Sequential(*[AttentionBlock(embed_dim, hidden_dim, num_heads, dropout=dropout) for _ in range(num_layers)])
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes),
            #nn.Softmax(dim=1)
        )
        self.dropout = nn.Dropout(dropout)

        # Parameters/Embeddings
        self.cls_token = nn.Parameter(torch.randn(1,1,embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1,1+num_patches,embed_dim))


    def forward(self, x):
        # Preprocess input
        #print("Input: ", x.shape)
        x = img_to_patch(x, self.patch_size)
        #print("Patches: ", x.shape)
        B, T, _ = x.shape
        x = self.input_layer(x)
        #print("Linear1: ", x.shape)

        # Add CLS token and positional encoding
        cls_token = self.cls_token.repeat(B, 1, 1)
        x = torch.cat([cls_token, x], dim=1)
        #print("Classification Token: ", x.shape)
        x = x + self.pos_embedding[:,:T+1]
       # print("Positional Embedding: ", x.shape)

        # Apply Transforrmer
        x = self.dropout(x)
        x = x.transpose(0, 1)
        x = self.transformer(x)

        # Perform classification prediction
        cls = x[0]
        out = self.mlp_head(cls)
        #print(out.shape)
        return out

In [8]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=5):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        for phase in ['train', 'val']: 
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    #_, preds = torch.max(outputs, 1) # used for accuracy
                    preds = torch.argmax(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()

    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [9]:

 #train
transform_train = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.25),
    #transforms.Resize(128),
    #transforms.CenterCrop(128),
    transforms.ToTensor(),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
    #transforms.RandomErasing(p=0.2, value='random')
])
transform_val = transforms.Compose([
    #transforms.Resize(128),
    #transforms.CenterCrop(128),
    transforms.ToTensor(),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
    
train_data = ImageFolderCustom(root='../input/iais22-birds/birds/birds',setname='train', transform = transform_train)
train_loader = DataLoader(train_data, batch_size=256, shuffle=True, num_workers=4)
train_data_len = len(train_data)

val_data = ImageFolderCustom(root='../input/iais22-birds/birds/birds',setname='val', transform = transform_val)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=4)
valid_data_len = len(val_data)

print(f"Found {len(train_data)} images for training with {len(train_data.classes)} classes")
print(f"Found {len(val_data)} images for validation with {len(val_data.classes)} classes")

final_train_data = datasets.ImageFolder(root='../input/iais22-birds/birds/birds', transform = transform_train)
final_train_loader = DataLoader(final_train_data, batch_size=256, shuffle=True, num_workers=4)
final_train_data_len = len(final_train_data)

print(f"Found {len(final_train_data)} images for final training with {len(final_train_data.classes)} classes")

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found 52714 images for training with 400 classes
Found 5674 images for validation with 400 classes
Found 58388 images for final training with 400 classes


In [10]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

final_dataloaders = {
    "train": final_train_loader,
    "val": val_loader
}
final_dataset_sizes = {
    "train": final_train_data_len,
    "val": valid_data_len
}

In [11]:
classes = get_classes("../input/iais22-birds/birds/birds")

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = True

''' 
    embed_dim - Dimensionality of the input feature vectors to the Transformer
    hidden_dim - Dimensionality of the hidden layer in the feed-forward networks
                         within the Transformer
    num_channels - Number of channels of the input (3 for RGB)
    num_heads - Number of heads to use in the Multi-Head Attention block
    num_layers - Number of layers to use in the Transformer
    num_classes - Number of classes to predict
    patch_size - Number of pixels that the patches have per dimension
    num_patches - Maximum number of patches an image can have
    dropout - Amount of dropout to apply in the feed-forward network and
            on the input encoding
''' 

model = VisionTransformer(embed_dim = 1024, hidden_dim = 2048, num_channels = 3,
                           num_heads = 16, num_layers = 3, num_classes = 400, patch_size = 16, num_patches = 256, dropout=0.2)
model = model.to(device)
#print(model.classifier)


In [13]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.11).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0003)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [14]:

#model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, dataloaders, dataset_sizes, num_epochs=7)
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, final_dataloaders, final_dataset_sizes, num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/229 [00:04<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 198.00 MiB (GPU 0; 15.90 GiB total capacity; 14.66 GiB already allocated; 169.75 MiB free; 14.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
image_list = []
preds_id = []
for filename in glob.glob("../input/iais22-birds/submission_test/submission_test/*.jpg"): 
    im=Image.open(filename)
    id = os.path.basename(filename).split(".")[0]
    image_list.append(im)
    preds_id.append(id)

index=[]
preds = []
for f in image_list:
    i = image_list.index(f)+1
    predict_class = pre_image(f,model)
    index.append(i)
    preds.append(predict_class)
    if(i%500==0):
        print(i)

submission = pd.DataFrame(
    data =np.array([preds_id,preds ]).T, 
    columns = ["Id", "Category"]
)
submission.to_csv("submission.csv", index = False)
submission.head()